In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_279680\1102906597.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Read file with largest interval

In [3]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_11520s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [4]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

### Clean elements that have a larger associated element

In [5]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-744.679039,-95.060790,1,3,1827.657345,3809.082584,0.335591,0.570318,1,-744.679039,-1161,105.836138,-888.242242,21.967109,-72.768028,0.762949
1,2,1,1,NaN,-412.025671,-77.163422,0,2,1877.470658,3809.470658,0.499138,0.499138,1,-412.025671,"-1160,9995",80.715163,-888.407168,16.705742,-72.902785,0.508633
2,3,1,1,NaN,-935.759582,-95.803365,1,4,1882.287636,3807.993468,0.261344,0.392628,1,-935.759582,"-1165,-1164,-1163,-1160,9995",78.286886,-887.659248,16.083357,-72.790183,1.017266
3,4,1,1,NaN,-466.020419,-93.020869,1,2,1875.444084,3805.000000,0.496864,0.000000,1,-466.020419,"-1160,-6",81.739922,-886.153878,16.568420,-72.535443,0.508633
4,5,1,1,NaN,-443.545833,-95.669428,1,2,1884.596392,3804.000000,0.490621,0.000000,1,-443.545833,"-1165,-1163,-1162",77.125053,-885.643926,15.550300,-72.469990,0.508633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214546,1156490,97,8,NaN,90.790462,25.972214,0,2,81.512523,343.000000,0.499843,0.000005,1,NaN,"1045070,1069963",-706.931292,354.013425,-52.025698,19.806140,32.632991
1214547,1156491,97,8,NaN,101.420263,32.749301,0,3,225.643491,349.645185,0.338682,0.338320,0,NaN,None,-124.733243,380.855776,-8.005519,20.263683,48.949487
1214548,1156492,97,8,NaN,266.792297,30.131560,0,4,67.502455,359.048986,0.288672,0.413092,1,NaN,1119615,-763.523115,418.841190,-61.813500,24.571732,65.265982
1214549,1156493,97,8,NaN,66.912197,29.341155,0,2,195.554315,386.000000,0.497041,0.000005,1,NaN,1156458,-246.274504,527.706259,-17.420963,30.451188,32.632991


### Calculate lifetimes

In [6]:
# Add life column
tabs_NoassLrg['Life'] = 0

# Calculate lifes
unique_id = np.unique(tabs_NoassLrg['id'])
for i in tqdm(np.arange(0, unique_id.shape[0])):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == unique_id[i]].copy().reset_index(drop=True)
    tabs_NoassLrg.loc[tabs_NoassLrg['id'] == unique_id[i],'Life'] = (np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1)



100%|██████████| 855472/855472 [2:37:19<00:00, 90.62it/s]   


### Drop single frame objects and change frame to time

In [7]:
tabs_NoassLrg.to_pickle('data\SWAMIS\\100_frames\swamis_NoAssociate_11520s.pnds')